In [1]:
pip install elasticsearch requests numpy sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [2]:
import types
import pandas as pd
import ibm_boto3
from botocore.client import Config
import json

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.

Cloud_Object_Storage_cx_client = ibm_boto3.client(
    service_name='s3',
    ibm_api_key_id='ADD YOUR API KEY HERE',
    ibm_service_instance_id='crn:v1:bluemix:public:cloud-object-storage:global:a/255bb8163fa943e89de4555de2a7d13c:2a47ec7f-8695-42ae-bedd-8b091e397eea::',
    ibm_auth_endpoint='https://iam.cloud.ibm.com/identity/token',
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.us-south.cloud-object-storage.appdomain.cloud'
)
bucket = '007tlhdata'
object_key = 'json_customer_data/all_customers_data.json'

body = Cloud_Object_Storage_cx_client.get_object(Bucket=bucket, Key=object_key)['Body'].read()


# Decode the body into JSON
data = json.loads(body)


all_customer_data = json.loads(body)


In [3]:
import tempfile
from elasticsearch import Elasticsearch
import math
from elasticsearch import Elasticsearch


# Create an Elasticsearch client
es = Elasticsearch(
    "https://93c0666f-8d7e-4e02-8033-be74f8b98de7.c5km1ted03t0e8geevf0.databases.appdomain.cloud:31018",
    verify_certs=False,
    basic_auth=("ibm_cloud_4a5c6b85_9b6b_4332_9b9b_03278e2ed106", "35eda2f56e83b1c775a50e98cc711dfd1c7533b5f23232086eea2285f774f5fe")
)


/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/elasticsearch/_sync/client/__init__.py:399: SecurityWarning: Connecting to 'https://93c0666f-8d7e-4e02-8033-be74f8b98de7.c5km1ted03t0e8geevf0.databases.appdomain.cloud:31018' using TLS with verify_certs=False is insecure
  _transport = transport_class(


In [4]:
from elasticsearch import Elasticsearch, helpers
from sentence_transformers import SentenceTransformer
import numpy as np

# Define index name
index_name = "customer_portfolios"

# Step 1: Create Index with Mapping
def create_index():
    mapping = {
        "mappings": {
            "properties": {
                "Customer ID": {"type": "keyword"},
                "Executive Summary": {
                    "properties": {
                        "Customer Profile": {
                            "properties": {
                                "Risk Tolerance": {"type": "keyword"},
                                "Preferred Sectors": {"type": "text"},
                                "Investment Goals": {"type": "keyword"}
                            }
                        },
                        "Portfolio Summary": {
                            "properties": {
                                "Total Current Value": {"type": "double"},
                                "Net Unrealized Gain/Loss": {"type": "double"}
                            }
                        }
                    }
                },
                "Portfolio Summary": {
                    "properties": {
                        "Total Initial Investment": {"type": "double"},
                        "Total Current Value": {"type": "double"},
                        "Net Unrealized Gain/Loss Percent": {"type": "double"}
                    }
                },
                "combined_embedding": {
                    "type": "dense_vector",
                    "dims": 384  # Matches the embedding dimensions
                }
            }
        }
    }

    if not es.indices.exists(index=index_name):
        es.indices.create(index=index_name, body=mapping)
        print(f"Index '{index_name}' created successfully!")
    else:
        print(f"Index '{index_name}' already exists!")

# Step 2: Generate Embeddings
def generate_embedding(text):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embedding = model.encode(text)
    return embedding.tolist()  # Convert to list for Elasticsearch

# Step 3: Ingest Data
def ingest_data(data):
    for record in data:
        customer_id = record["Customer ID"]

        # Combine text fields for embedding generation
        combined_text = (
            str(record["Executive Summary"]) +
            str(record.get("Portfolio Summary", "")) +
            str(record.get("Tax Loss Harvesting Analysis", "")) +
            str(record.get("Reinvestment Recommendations", ""))
        )

        # Generate embeddings
        embedding = generate_embedding(combined_text)

        # Add embedding to the record
        record["combined_embedding"] = embedding

        # Index the document
        es.index(index=index_name, id=customer_id, body=record)

    print("Data ingested successfully!")

# Main Execution
if __name__ == "__main__":
    all_customer_data = json.loads(body)
    create_index()
    ingest_data(all_customer_data)


/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '93c0666f-8d7e-4e02-8033-be74f8b98de7.c5km1ted03t0e8geevf0.databases.appdomain.cloud'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '93c0666f-8d7e-4e02-8033-be74f8b98de7.c5km1ted03t0e8geevf0.databases.appdomain.cloud'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Index 'customer_portfolios' created successfully!


/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '93c0666f-8d7e-4e02-8033-be74f8b98de7.c5km1ted03t0e8geevf0.databases.appdomain.cloud'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Data ingested successfully!


/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '93c0666f-8d7e-4e02-8033-be74f8b98de7.c5km1ted03t0e8geevf0.databases.appdomain.cloud'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
